<a href="https://colab.research.google.com/github/KenObata/Music4All_personal_project/blob/main/week15_ALBERT_large_baseline_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Week15: This notebook uses Pre-Trained word matrix-> SMOTE -> undersample by EDA -> append to word vectors. -> using Word2Vec. After that I apply SMOTE to balance out. 
Added task: Grid Search for best parameter in SVM.

Situation: English only (=multi-class).
Split: StratifiedKfold.
Reference: https://github.com/jasonwei20/eda_nlp


### set up

In [1]:
!pip3 install scikit-multilearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 89 kB 4.1 MB/s 


In [2]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn
from collections import Counter

from skmultilearn.model_selection import IterativeStratification   
from sklearn.model_selection import StratifiedKFold
from scipy.sparse import csr_matrix

In [3]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import balanced_accuracy_score
DIR = '/content/drive/MyDrive/music4all/'
def get_balanced_accuracy(model, McNemar, is_fine_tuning, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning):
  test_y = test.map(map_func_only_y)
  y_category=np.zeros((TEST_SIZE, ))
  counter=0
  for label_tensor in test_y.take(len(test_y)):
    y_test = np.argmax(label_tensor, axis=1)
    for label in y_test:
      y_category[counter]=label
      counter+=1

  X_test, y_test = test.map(map_func_only_X), y_category
  y_predict_test = np.asarray(model.predict(X_test))
  y_predict_test = np.argmax(y_predict_test, axis=1)
  print(classification_report(y_test, y_predict_test) )
  print(balanced_accuracy_score(y_test, y_predict_test))

  McNemar[(is_fine_tuning, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning)] = []
  for ground_truh, pred in zip(y_test, y_predict_test):
        if ground_truh==pred:
          McNemar[(is_fine_tuning, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning)].append(True)
        else:
          McNemar[(is_fine_tuning, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning)].append(False)
  with open(DIR+ "ALBERT_large_log.txt", "a") as f:
    print("======================================", file=f)
    print("is_fine_tuning?:", is_fine_tuning, "drop_out_rate: ", drop_out_rate, "learning_rate_transfer_learning: ", learning_rate_transfer_learning,
          "learning_rate_fine_tuning: ", learning_rate_fine_tuning, file=f)
    print(classification_report(y_test, y_predict_test) , file=f)
    print(balanced_accuracy_score(y_test, y_predict_test), file=f)

  return balanced_accuracy_score(y_test, y_predict_test), McNemar

### Data Preparation(Kfold split)

Create dataframe for Gensim

In [4]:
import pandas as pd

DIR = '/content/drive/MyDrive/music4all/'
df_genre_by_lang = pd.read_csv(DIR + 'df_genre_by_lang_full.csv')
df_genre_by_lang

,Unnamed: 0,id,genres,lang,lyric,number_of_line
0,0,0009fFIM1eYThaPg,pop,en,a sunny day so I got nowhere to hide Not a clo...,91
1,1,00P2bHdWFkghmDqz,soul,en,Tell me a tale that always was Sing me a song ...,36
2,2,00b6fV3nx5z2b8Ls,pop,en,A buh A buh You went to school to learn girl T...,74
3,3,013QDoTqbexEwkHr,pop,en,like a conversation where stops to breathe Is ...,20
4,4,01EKNot8qVgZpKM7,rock,en,Say the words I cannot say Say them on another...,31
...,...,...,...,...,...,...
13535,13535,zzT504Z94j1IAuc3,indie rock,en,think what afraid of come in you know been mad...,18
13536,13536,zzgS4ZqyswamEWNj,pop,en,Oh yeah yeah Last night I took a walk in the s...,75
13537,13537,zzx8CWdM7qkxKQpC,indie rock,en,Innocence it come easy in a sense it never wil...,34
13538,13538,zzz0n04uuTUA7fNh,pop,en,Girl you know how I feel I really Since you be...,65


In [5]:
def load_data(df_col, y):
    texts, labels = [], []
    
    for line in df_col:
        # texts are already tokenized, just split on space
        # in a real use-case we would put more effort in preprocessing
        texts.append(line.split(' '))
    return pd.DataFrame({'texts': texts, 'labels': y})

In [6]:
data = load_data(df_genre_by_lang["lyric"], df_genre_by_lang["genres"])

In [7]:
data

,texts,labels
0,"[a, sunny, day, so, I, got, nowhere, to, hide,...",pop
1,"[Tell, me, a, tale, that, always, was, Sing, m...",soul
2,"[A, buh, A, buh, You, went, to, school, to, le...",pop
3,"[like, a, conversation, where, stops, to, brea...",pop
4,"[Say, the, words, I, cannot, say, Say, them, o...",rock
...,...,...
13535,"[think, what, afraid, of, come, in, you, know,...",indie rock
13536,"[Oh, yeah, yeah, Last, night, I, took, a, walk...",pop
13537,"[Innocence, it, come, easy, in, a, sense, it, ...",indie rock
13538,"[Girl, you, know, how, I, feel, I, really, Sin...",pop


In [8]:
data['labels'] = data['labels'].astype('category')
label_mapping = data['labels'].cat.categories
data['labels'] = data['labels'].cat.codes
X = data['texts']
y = data['labels']

In [9]:
type(X)

pandas.core.series.Series

In [10]:
def StratifiedKFold_feature_and_df_glove(df, feature_list, y_name):
  skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1209)  # 20% for test set 
  y = df[y_name]
  skf.get_n_splits(df[ feature_list ], y)

  splits = []

  for train_index, test_index in skf.split(df[ feature_list ], y):
      print("TRAIN:", train_index, "TEST:", test_index)
      X_train, X_test = df[ feature_list ].loc[train_index], df[ feature_list ].loc[test_index]
      y_train, y_test = y.loc[train_index], y.loc[test_index]
      splits.append({'X_train': X_train, 'X_test': X_test, 'y_train':y_train, 'y_test':y_test })
  return splits

In [11]:
def StratifiedKFold_feature_and_df(X, y):
  skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1209)  # 20% for test set 
  #y = df[y_name]
  skf.get_n_splits(X, y)#df[ feature_list ]

  splits = []

  for train_index, test_index in skf.split(X, y):#df[ feature_list ]
      print("TRAIN:", train_index, "TEST:", test_index)
      X_train, X_test = X.loc[train_index], X.loc[test_index]
      y_train, y_test = y.loc[train_index], y.loc[test_index]
      splits.append({'X_train': X_train, 'X_test': X_test, 'y_train':y_train, 'y_test':y_test })
  return splits

In [12]:
#feature_list = ["texts"] #this is BOW and TF-IDF
#splits = StratifiedKFold_feature_and_df( data, feature_list, 'labels')
splits = StratifiedKFold_feature_and_df( X,y)

TRAIN: [    0     1     3 ... 13537 13538 13539] TEST: [    2     4     5 ... 13526 13532 13535]
TRAIN: [    0     2     4 ... 13535 13536 13539] TEST: [    1     3     7 ... 13530 13537 13538]
TRAIN: [    0     1     2 ... 13537 13538 13539] TEST: [    8    14    22 ... 13521 13531 13536]
TRAIN: [    0     1     2 ... 13537 13538 13539] TEST: [   10    12    15 ... 13523 13525 13534]
TRAIN: [    1     2     3 ... 13536 13537 13538] TEST: [    0     6    11 ... 13529 13533 13539]


In [13]:
len(splits)

5

In [14]:
split0=splits[0]
print(split0['X_train'].shape)
print(split0['y_train'].shape)
print(split0['X_test'].shape)
print(split0['y_test'].shape)

(10832,)
(10832,)
(2708,)
(2708,)


In [15]:
split0['X_train']

0        [a, sunny, day, so, I, got, nowhere, to, hide,...
1        [Tell, me, a, tale, that, always, was, Sing, m...
3        [like, a, conversation, where, stops, to, brea...
6        [Locked, up, tight, Like, I, would, never, fee...
7        [sittin, in, the, crib, dreamin, about, leer, ...
                               ...                        
13534    [grandma, cookies, nigga, Shout, out, to, fron...
13536    [Oh, yeah, yeah, Last, night, I, took, a, walk...
13537    [Innocence, it, come, easy, in, a, sense, it, ...
13538    [Girl, you, know, how, I, feel, I, really, Sin...
13539    [wwI, oh, must, go, on, standing, You, break, ...
Name: texts, Length: 10832, dtype: object

In [16]:
split0['y_train']

0        4
1        9
3        4
6        4
7        6
        ..
13534    6
13536    4
13537    3
13538    4
13539    8
Name: labels, Length: 10832, dtype: int8

### Use my self programmed balanced accuracy

In [ ]:
metrics = Metrics()
history = model.fit(train, validation_data=val, epochs=10, class_weight=my_weight ,callbacks=[metrics])
metrics.get_data()

Epoch 1/10
339/339 [==============================] - 225s 645ms/step - loss: 0.5565 - categorical_accuracy: 0.3878 - val_loss: 1.9337 - val_categorical_accuracy: 0.3273
Epoch 2/10
339/339 [==============================] - 218s 645ms/step - loss: 0.5397 - categorical_accuracy: 0.3626 - val_loss: 2.0651 - val_categorical_accuracy: 0.2764
Epoch 3/10
339/339 [==============================] - 218s 645ms/step - loss: 0.5375 - categorical_accuracy: 0.3392 - val_loss: 2.0931 - val_categorical_accuracy: 0.2273
Epoch 4/10
339/339 [==============================] - 218s 645ms/step - loss: 0.5378 - categorical_accuracy: 0.3197 - val_loss: 2.0576 - val_categorical_accuracy: 0.2459
Epoch 5/10
339/339 [==============================] - 219s 645ms/step - loss: 0.5333 - categorical_accuracy: 0.3077 - val_loss: 2.0292 - val_categorical_accuracy: 0.2816
Epoch 6/10
339/339 [==============================] - 219s 646ms/step - loss: 0.5373 - categorical_accuracy: 0.2967 - val_loss: 2.1738 - val_categoric

[{'balanced_accuracy': 0.09744849916749872},
 {'balanced_accuracy': 0.10420636932765996},
 {'balanced_accuracy': 0.10970405736872939},
 {'balanced_accuracy': 0.10356561320774078},
 {'balanced_accuracy': 0.10792864804613309},
 {'balanced_accuracy': 0.09168989857066105},
 {'balanced_accuracy': 0.09926135499834263},
 {'balanced_accuracy': 0.10014630125778516},
 {'balanced_accuracy': 0.09807505457274496},
 {'balanced_accuracy': 0.09881853716030842}]

## From here, separate X_train, X_test from KFOldSplit

### Preprocess my lyrics data (Official train and test)

In [17]:
import tensorflow as tf
!pip3 install transformers
SEQ_LEN = 256#512

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.1 MB/s 
     |████████████████████████████████| 120 kB 90.4 MB/s 
     |████████████████████████████████| 6.6 MB 62.7 MB/s 


In [18]:
split0['X_test']

2        [A, buh, A, buh, You, went, to, school, to, le...
4        [Say, the, words, I, cannot, say, Say, them, o...
5        [I, was, alone, I, was, made, of, stone, You, ...
9        [Again, the, burden, of, losing, rests, upon, ...
20       [only, been, three, weeks, And, a, bag, of, sp...
                               ...                        
13517    [Like, the, legend, of, the, Phoenix, All, end...
13522    [Mr, Telephone, man, something, wrong, with, m...
13526    [can, you, imagine, what, it, would, be, like,...
13532    [Love, of, my, life, hurt, me, broken, my, hea...
13535    [think, what, afraid, of, come, in, you, know,...
Name: texts, Length: 2708, dtype: object

In [19]:
def prepare_lyrics(X_series):
  for i, token_list in X_series.items():
    if type(token_list) is list:
      X_series.loc[i] = ' '.join(token_list)
  return X_series

In [20]:
split0['X_test'] = prepare_lyrics(split0['X_test'] )
split0['X_test']

2        A buh A buh You went to school to learn girl T...
4        Say the words I cannot say Say them on another...
5        I was alone I was made of stone You took me ho...
9        Again the burden of losing rests upon my shoul...
20       only been three weeks And a bag of speed from ...
                               ...                        
13517    Like the legend of the Phoenix All ends with b...
13522    Mr Telephone man something wrong with my line ...
13526    can you imagine what it would be like we never...
13532    Love of my life hurt me broken my heart and no...
13535    think what afraid of come in you know been mad...
Name: texts, Length: 2708, dtype: object

In [21]:
split0['X_train'] = prepare_lyrics(split0['X_train'] )
split0['X_train'] 

0        a sunny day so I got nowhere to hide Not a clo...
1        Tell me a tale that always was Sing me a song ...
3        like a conversation where stops to breathe Is ...
6        Locked up tight Like I would never feel again ...
7        sittin in the crib dreamin about leer jets and...
                               ...                        
13534    grandma cookies nigga Shout out to fronto leaf...
13536    Oh yeah yeah Last night I took a walk in the s...
13537    Innocence it come easy in a sense it never wil...
13538    Girl you know how I feel I really Since you be...
13539    wwI oh must go on standing You break that whic...
Name: texts, Length: 10832, dtype: object

In [22]:
split0['y_train'] 

0        4
1        9
3        4
6        4
7        6
        ..
13534    6
13536    4
13537    3
13538    4
13539    8
Name: labels, Length: 10832, dtype: int8

use ALBERT
Be careful, you need sentencepiece library:https://stackoverflow.com/questions/65854722/huggingface-albert-tokenizer-nonetype-error-with-colab

In [23]:
!pip3 install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.0 MB/s 


In [24]:
from transformers import AutoTokenizer
from transformers import AlbertTokenizer, TFAlbertModel

#tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
tokenizer = AlbertTokenizer.from_pretrained('albert-large-v2')

In [25]:
import numpy as np
Xids_train = np.zeros((split0['X_train'].shape[0], SEQ_LEN))
Xmask_train = np.zeros((split0['X_train'].shape[0], SEQ_LEN))
Xids_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))
Xmask_test = np.zeros((split0['X_test'].shape[0], SEQ_LEN))

for i, lyric in enumerate(split0['X_train']):
  tokens = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length", add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
  Xids_train[i,:], Xmask_train[i,:] = tokens['input_ids'], tokens['attention_mask']

for i, lyric in enumerate(split0['X_test']):
  tokens_test = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length"
    , add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
  Xids_test[i,:], Xmask_test[i,:] = tokens_test['input_ids'], tokens_test['attention_mask']
print(Xids_train.shape, Xids_test.shape)

(10832, 256) (2708, 256)


Create y labels in transformer format

In [26]:
np.set_printoptions(threshold=np.inf)

In [27]:
labels_train = np.zeros((split0['y_train'].shape[0], 10))
labels_train[ np.arange(split0['y_train'].shape[0]), split0['y_train'].values] =1
labels_test = np.zeros((split0['y_test'].shape[0], 10))
labels_test[ np.arange(split0['y_test'].shape[0]), split0['y_test'].values] =1

dataset_train = tf.data.Dataset.from_tensor_slices((Xids_train, Xmask_train, labels_train))
dataset_test = tf.data.Dataset.from_tensor_slices((Xids_test, Xmask_test, labels_test))

In [28]:
len(dataset_train)

10832

In [29]:
def map_func(input_ids, masks, labels):
  return {'input_ids': input_ids, 'attention_mask':masks}, labels

In [30]:
dataset_train = dataset_train.map(map_func)
dataset_test = dataset_test.map(map_func)

In [31]:
dataset_train = dataset_train.shuffle(42).batch(16)
DS_LEN = len(list(dataset_train))
DS_LEN

677

339 because 10800/32

In [32]:
SPLIT = 0.9
round(DS_LEN*SPLIT)

609

In [33]:
len(split0['X_train'])-int(len(split0['X_train'])*0.9)

1084

In [34]:
train = dataset_train.take(round(DS_LEN*SPLIT))
val = dataset_train.skip(round(DS_LEN*SPLIT))#remainer is for val.

In [35]:
test = dataset_test.batch(16)

In [36]:
def map_func_only_X(val_dictionary, labels):
  return {'input_ids': val_dictionary['input_ids'], 'attention_mask':val_dictionary['attention_mask']}

In [37]:
def map_func_only_y(val_dictionary, labels):
  return labels

In [38]:
VAL_SIZE=1084

In [39]:
counter = Counter(split0['y_train'])
SUM=0
for item in list(counter.values()) :
  SUM+=item
#SUM = sum(counter.values())
SUM

10832

In [40]:
#Tutorial
#weight_for_0 = (1 / neg) * (total / 2.0)
#weight_for_1 = (1 / pos) * (total / 2.0)

counter = Counter(split0['y_train'])
my_weight2 = {}
print(counter)

for genre in counter:
  #print(genre, counter[genre])
  my_weight2[genre] = (1/counter[genre]) * (SUM/10)
my_weight2

Counter({4: 4143, 7: 1159, 9: 1030, 3: 865, 6: 783, 0: 763, 1: 690, 8: 556, 2: 537, 5: 306})


{4: 0.2614530533429882,
 9: 1.051650485436893,
 6: 1.383397190293742,
 2: 2.0171322160148977,
 0: 1.4196592398427261,
 7: 0.9345987920621226,
 1: 1.569855072463768,
 5: 3.539869281045752,
 8: 1.948201438848921,
 3: 1.2522543352601156}

In [41]:
TEST_SIZE = split0["y_test"].shape[0]
TEST_SIZE

2708

### Imbalance

In [ ]:
MAX = max(counter.values())
MAX

4143

In [ ]:
my_weight={}

for genre in range(10):
  my_weight[genre]=MAX/counter[genre]
#sum(weight)
my_weight

{0: 5.4298820445609435,
 1: 6.004347826086956,
 2: 7.715083798882682,
 3: 4.789595375722543,
 4: 1.0,
 5: 13.53921568627451,
 6: 5.291187739463601,
 7: 3.5746333045729077,
 8: 7.451438848920863,
 9: 4.022330097087378}

In [ ]:
from transformers import TFAutoModel
bert = TFAutoModel.from_pretrained('bert-base-cased')

input_ids = tf.keras.layers.Input(shape=(SEQ_LEN,), name= 'input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(SEQ_LEN,), name='attention_mask')

embeddings = bert(input_ids, attention_mask= mask)[0]
X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(128, activation = 'relu')(X)
X = tf.keras.layers.Dropout(0.1)(X)
X = tf.keras.layers.Dense(32, activation = 'relu')(X)
y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
model.layers[2].trainable = False
model.summary()

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 512)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                           

BERT is huge, so here we don't fine tune BERT. Simply use pre-trained base BERT

In [ ]:
optimizer = tf.keras.optimizers.Adam(1e-3)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model.compile(optimizer=optimizer, loss= loss, metrics=metrics)

In [ ]:
print(DS_LEN, len(train), len(val), len(test))

339 305 34 85


In [ ]:
VAL_SIZE = len(val)*32
VAL_SIZE

1088

In [ ]:
#print(len(val.take(len(val))))
for item, label_batch in val.take(1):#len(val)
  print(item['input_ids'])
  #one_batch= item[0]['input_ids']
  #for xids in one_batch:
  #  print(xids.shape)

In [ ]:
val_X = val.map(map_func_only_X)
print(len(val_X))
#for i in val_X.take(1):
  #print(i)

34


In [ ]:
val_y = val.map(map_func_only_y)
print(len(val_y))
y_category=np.zeros((VAL_SIZE, ))
counter=0
for label_tensor in val_y.take(1):
  #print(label_tensor)
  y_val = np.argmax(label_tensor, axis=1)
  for label in y_val:
    print(label) 
    y_category[counter]=label
    counter+=1
print('counter:',counter)
print(y_category)

In [ ]:
VAL_SIZE=1072
VAL_SIZE

1072

In [ ]:
VAL_SIZE=len(split0['X_train'])-int(len(split0['X_train'])*0.9)
VAL_SIZE

1084

In [ ]:
val_y = val2.map(map_func_only_y)
print(len(val_y))
y_category=np.zeros((VAL_SIZE, ))
counter=0
for label_tensor in val_y.take(len(val2)):
  #print(label_tensor)
  y_val = np.argmax(label_tensor, axis=1)
  for label in y_val:
    print(label) 
    y_category[counter]=label
    counter+=1
print('counter:',counter)
print(y_category)

Ref1: how to calculate genre as int instead of one hot vector from list of one hot vectors.
https://www.sharpsightlabs.com/blog/numpy-argmax/

Ref2: How to use self-programmed metrics: https://stackoverflow.com/questions/37657260/how-to-implement-custom-metric-in-keras



In [ ]:
VAL_SIZE=1072

In [ ]:
from sklearn.metrics import balanced_accuracy_score
class Metrics(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self._data = []
        self.validation_data = val2

    def map_func_only_X(val_dictionary, labels):
      return {'input_ids': val_dictionary['input_ids'], 'attention_mask':val_dictionary['attention_mask']}
    def map_func_only_y(val_dictionary, labels):
      return labels

    def on_epoch_end(self, batch, logs={}):
      #Xid_list= np.zeros((VAL_SIZE, SEQ_LEN))
      #y_onehot_list= np.zeros((VAL_SIZE, 10))
      #counter=0
      #for item in dataset.take(len(self.validation_data)):
      #for item in val.take(len(val)):
      """
      for item, batch_label in self.validation_data.take(len(val)):
        for label in batch_label:
          y_onehot_list[counter,:] = label
          counter+=1
        one_batch_xid = item['input_ids']
        for xids in one_batch_xid:
          Xid_list[counter, :] = xids
        
        #X_val, y_val = self.validation_data[0], self.validation_data[1]
      """
      val_y = self.validation_data.map(map_func_only_y)
      y_category=np.zeros((VAL_SIZE, ))
      counter=0
      for label_tensor in val_y.take(len(val_y)):
        y_val = np.argmax(label_tensor, axis=1)
        for label in y_val:
          y_category[counter]=label
          counter+=1
      X_val, y_val =self.validation_data.map(map_func_only_X), y_category
      y_predict = np.asarray(model2.predict(X_val))

      #y_val = np.argmax(y_val, axis=1)#I have already done converting to category
      y_predict = np.argmax(y_predict, axis=1)

      self._data.append({
          'balanced_accuracy': balanced_accuracy_score(y_val, y_predict),
          #'val_rocauc': roc_auc_score(y_val, y_predict),
      })
      return

    def get_data(self):
        return self._data

### Transfer learning: try early stopping with SEQ_LEN=512

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)

metrics = Metrics()
history = model.fit(train, validation_data=val, epochs=10, class_weight=my_weight ,callbacks=[metrics, early_stopping])
metrics.get_data()

Epoch 1/10
305/305 [==============================] - 165s 541ms/step - loss: 7.4384 - categorical_accuracy: 0.2784 - val_loss: 2.1002 - val_categorical_accuracy: 0.2118
Epoch 2/10
305/305 [==============================] - 165s 541ms/step - loss: 7.0577 - categorical_accuracy: 0.3053 - val_loss: 2.0583 - val_categorical_accuracy: 0.2435
Epoch 3/10
305/305 [==============================] - 165s 542ms/step - loss: 6.7958 - categorical_accuracy: 0.3207 - val_loss: 1.9879 - val_categorical_accuracy: 0.2677
Epoch 4/10
305/305 [==============================] - 165s 542ms/step - loss: 6.5908 - categorical_accuracy: 0.3376 - val_loss: 2.0138 - val_categorical_accuracy: 0.2388
Epoch 5/10
305/305 [==============================] - 165s 542ms/step - loss: 6.4173 - categorical_accuracy: 0.3493 - val_loss: 2.0363 - val_categorical_accuracy: 0.2472
Epoch 6/10
305/305 [==============================] - 165s 542ms/step - loss: 6.2501 - categorical_accuracy: 0.3629 - val_loss: 2.0147 - val_categoric

[{'balanced_accuracy': 0.10064757124688836},
 {'balanced_accuracy': 0.1187677795399048},
 {'balanced_accuracy': 0.09990412917019217},
 {'balanced_accuracy': 0.08602748843296298},
 {'balanced_accuracy': 0.12293896335258817},
 {'balanced_accuracy': 0.08466265273217208}]

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import balanced_accuracy_score
test_y = test.map(map_func_only_y)
y_category=np.zeros((TEST_SIZE, ))
counter=0
for label_tensor in test_y.take(len(test_y)):
  y_test = np.argmax(label_tensor, axis=1)
  for label in y_test:
    y_category[counter]=label
    counter+=1

X_test, y_test = test.map(map_func_only_X), y_category
y_predict_test = np.asarray(model.predict(X_test))
y_predict_test = np.argmax(y_predict_test, axis=1)
print(classification_report(y_test, y_predict_test) )
print(balanced_accuracy_score(y_test, y_predict_test))

              precision    recall  f1-score   support

         0.0       0.14      0.42      0.21       190
         1.0       0.16      0.28      0.20       173
         2.0       0.10      0.14      0.12       135
         3.0       0.13      0.17      0.15       216
         4.0       0.74      0.19      0.31      1036
         5.0       0.12      0.38      0.18        76
         6.0       0.77      0.77      0.77       195
         7.0       0.11      0.08      0.10       290
         8.0       0.09      0.09      0.09       139
         9.0       0.24      0.28      0.26       258

    accuracy                           0.25      2708
   macro avg       0.26      0.28      0.24      2708
weighted avg       0.42      0.25      0.26      2708

0.2825509565558356


# Transfer learning -> Fine tunine: unfreeze middle layers and train with smaller learning rate. using ALBERT base

In [42]:
SEQ_LEN2=256

In [48]:
from transformers import TFAutoModel
from transformers import AlbertTokenizer, TFAlbertModel
albert_base = TFAlbertModel.from_pretrained('albert-large-v2')

drop_out_rates = [0.1, 0.2, 0.3,0.4, 0.5]
balanced_accuracies_transfer_learning = []
balanced_accuracies_fine_tuning = []
for drop_out_rate in drop_out_rates:
  #step1
  input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
  mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

  embeddings = albert_base(input_ids, attention_mask= mask)[0]
  X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
  X = tf.keras.layers.BatchNormalization()(X)
  X = tf.keras.layers.Dense(128, activation = 'relu')(X)
  X = tf.keras.layers.Dropout(drop_out_rate)(X)
  X = tf.keras.layers.Dense(32, activation = 'relu')(X)
  y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

  model2 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
  model2.layers[2].trainable = False
  #model2.summary()

  #step2
  optimizer = tf.keras.optimizers.Adam(1e-3)
  loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
  metrics = []
  metrics.append(
      tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
  model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)
  #model2.summary() #Check trainable params increased.

  #step3: transfer learning
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
  history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

  #step4: predict
  balanced_acc=get_balanced_accuracy(model2)
  balanced_accuracies_transfer_learning.append( balanced_acc )

  #step5: fine tune
  model2.layers[2].trainable = True

  # It's important to recompile your model after you make any changes
  optimizer = tf.keras.optimizers.Adam(1e-5)
  loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
  metrics = []
  metrics.append(
      tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
  model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)

  history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
  balanced_acc=get_balanced_accuracy(model2)
  balanced_accuracies_fine_tuning.append( balanced_acc )

  del(model2)


Some layers from the model checkpoint at albert-large-v2 were not used when initializing TFAlbertModel: ['predictions']
- This IS expected if you are initializing TFAlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFAlbertModel were initialized from the model checkpoint at albert-large-v2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertModel for predictions without further training.


Epoch 1/6
609/609 [==============================] - 247s 369ms/step - loss: 2.0898 - categorical_accuracy: 0.2449 - val_loss: 1.9160 - val_categorical_accuracy: 0.2895
Epoch 2/6
609/609 [==============================] - 220s 361ms/step - loss: 1.6881 - categorical_accuracy: 0.3587 - val_loss: 1.8891 - val_categorical_accuracy: 0.3015
Epoch 3/6
609/609 [==============================] - 219s 360ms/step - loss: 1.4174 - categorical_accuracy: 0.4352 - val_loss: 1.8973 - val_categorical_accuracy: 0.3153
Epoch 4/6
609/609 [==============================] - 219s 360ms/step - loss: 1.1821 - categorical_accuracy: 0.4992 - val_loss: 2.0300 - val_categorical_accuracy: 0.2923
Epoch 5/6
609/609 [==============================] - 219s 360ms/step - loss: 0.9610 - categorical_accuracy: 0.5565 - val_loss: 2.1120 - val_categorical_accuracy: 0.2996
Epoch 6/6
609/609 [==============================] - 219s 360ms/step - loss: 0.8037 - categorical_accuracy: 0.6061 - val_loss: 2.1925 - val_categorical_acc

609/609 [==============================] - 570s 892ms/step - loss: 2.0808 - categorical_accuracy: 0.3358 - val_loss: 2.3407 - val_categorical_accuracy: 0.2739
Epoch 2/6
609/609 [==============================] - 539s 884ms/step - loss: 2.2728 - categorical_accuracy: 0.2945 - val_loss: 2.5517 - val_categorical_accuracy: 0.3079
Epoch 3/6
609/609 [==============================] - 539s 885ms/step - loss: 2.0363 - categorical_accuracy: 0.3161 - val_loss: 2.5536 - val_categorical_accuracy: 0.2390
Epoch 4/6
609/609 [==============================] - 538s 884ms/step - loss: 1.7753 - categorical_accuracy: 0.3454 - val_loss: 2.4465 - val_categorical_accuracy: 0.2417
Epoch 5/6
609/609 [==============================] - 538s 884ms/step - loss: 1.5539 - categorical_accuracy: 0.3792 - val_loss: 2.3367 - val_categorical_accuracy: 0.2665
              precision    recall  f1-score   support

         0.0       0.15      0.13      0.14       190
         1.0       0.15      0.38      0.21       173
  

609/609 [==============================] - 571s 891ms/step - loss: 2.3535 - categorical_accuracy: 0.2771 - val_loss: 4.3616 - val_categorical_accuracy: 0.0772
Epoch 2/6
609/609 [==============================] - 537s 882ms/step - loss: 1.9284 - categorical_accuracy: 0.3013 - val_loss: 3.9277 - val_categorical_accuracy: 0.0993
Epoch 3/6
609/609 [==============================] - 537s 882ms/step - loss: 1.8030 - categorical_accuracy: 0.3387 - val_loss: 3.6848 - val_categorical_accuracy: 0.1756
Epoch 4/6
609/609 [==============================] - 537s 883ms/step - loss: 1.7356 - categorical_accuracy: 0.3471 - val_loss: 2.4950 - val_categorical_accuracy: 0.2886
Epoch 5/6
609/609 [==============================] - 537s 881ms/step - loss: 1.9671 - categorical_accuracy: 0.3207 - val_loss: 5.9806 - val_categorical_accuracy: 0.0551
Epoch 6/6
609/609 [==============================] - 535s 879ms/step - loss: 2.1460 - categorical_accuracy: 0.2602 - val_loss: 7.0500 - val_categorical_accuracy: 0.0

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

609/609 [==============================] - 568s 892ms/step - loss: 2.0029 - categorical_accuracy: 0.3153 - val_loss: 5.8892 - val_categorical_accuracy: 0.1452
Epoch 2/6
609/609 [==============================] - 536s 880ms/step - loss: 1.8961 - categorical_accuracy: 0.3041 - val_loss: 2.9597 - val_categorical_accuracy: 0.2472
Epoch 3/6
609/609 [==============================] - 536s 881ms/step - loss: 1.8184 - categorical_accuracy: 0.3083 - val_loss: 2.1188 - val_categorical_accuracy: 0.2629
Epoch 4/6
609/609 [==============================] - 536s 881ms/step - loss: 1.7148 - categorical_accuracy: 0.2889 - val_loss: 4.4864 - val_categorical_accuracy: 0.1011
Epoch 5/6
609/609 [==============================] - 536s 881ms/step - loss: 1.6261 - categorical_accuracy: 0.2948 - val_loss: 2.2003 - val_categorical_accuracy: 0.2077
Epoch 6/6
609/609 [==============================] - 536s 881ms/step - loss: 1.5460 - categorical_accuracy: 0.2943 - val_loss: 1.9049 - val_categorical_accuracy: 0.2

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.13      0.43      0.20       190
         1.0       0.15      0.08      0.10       173
         2.0       0.06      0.13      0.08       135
         3.0       0.16      0.19      0.18       216
         4.0       0.48      0.15      0.22      1036
         5.0       0.23      0.24      0.23        76
         6.0       0.84      0.72      0.78       195
         7.0       0.00      0.00      0.00       290
         8.0       0.00      0.00      0.00       139
         9.0       0.19      0.65      0.29       258

    accuracy                           0.23      2708
   macro avg       0.22      0.26      0.21      2708
weighted avg       0.30      0.23      0.21      2708

0.2580569761133787
Epoch 1/6
609/609 [==============================] - 246s 367ms/step - loss: 1.6436 - categorical_accuracy: 0.3229 - val_loss: 1.9396 - val_categorical_accuracy: 0.2785
Epoch 2/6
609/609 [==============================] - 

609/609 [==============================] - 571s 891ms/step - loss: 1.4192 - categorical_accuracy: 0.3681 - val_loss: 2.9397 - val_categorical_accuracy: 0.2822
Epoch 2/6
609/609 [==============================] - 537s 882ms/step - loss: 1.3136 - categorical_accuracy: 0.3799 - val_loss: 4.1014 - val_categorical_accuracy: 0.1756
Epoch 3/6
609/609 [==============================] - 537s 882ms/step - loss: 1.2501 - categorical_accuracy: 0.3862 - val_loss: 3.1378 - val_categorical_accuracy: 0.1866
Epoch 4/6
609/609 [==============================] - 537s 882ms/step - loss: 1.1917 - categorical_accuracy: 0.4078 - val_loss: 3.2733 - val_categorical_accuracy: 0.2197
              precision    recall  f1-score   support

         0.0       0.10      0.73      0.18       190
         1.0       0.17      0.06      0.09       173
         2.0       0.05      0.13      0.07       135
         3.0       0.16      0.07      0.10       216
         4.0       0.61      0.12      0.21      1036
         

609/609 [==============================] - 567s 889ms/step - loss: 1.3344 - categorical_accuracy: 0.4152 - val_loss: 2.0446 - val_categorical_accuracy: 0.2748
Epoch 2/6
609/609 [==============================] - 536s 881ms/step - loss: 1.2897 - categorical_accuracy: 0.4080 - val_loss: 2.5091 - val_categorical_accuracy: 0.2592
Epoch 3/6
609/609 [==============================] - 537s 881ms/step - loss: 1.1043 - categorical_accuracy: 0.4618 - val_loss: 1.9240 - val_categorical_accuracy: 0.2702
Epoch 4/6
609/609 [==============================] - 537s 881ms/step - loss: 1.0043 - categorical_accuracy: 0.4783 - val_loss: 2.7361 - val_categorical_accuracy: 0.2537
              precision    recall  f1-score   support

         0.0       0.21      0.15      0.18       190
         1.0       0.17      0.12      0.14       173
         2.0       0.05      0.18      0.08       135
         3.0       0.22      0.14      0.17       216
         4.0       0.52      0.15      0.24      1036
         

In [51]:
balanced_accuracies_transfer_learning

[0.2714461328340759,
 0.2846293000154242,
 0.24044978540726597,
 0.2560964586279497,
 0.24175213805787257]

In [49]:
balanced_accuracies_fine_tuning

[0.27205025845354136,
 0.09203757848079876,
 0.2580569761133787,
 0.227055276787063,
 0.24536153069686137]

In [44]:
seen_parameters = []
drop_out_rates=[0.1,0.2,0.3,0.4,0.5]
for dropout in drop_out_rates:
  seen_parameters.append((1e-3, 1e-5, dropout))
seen_parameters

[(0.001, 1e-05, 0.1),
 (0.001, 1e-05, 0.2),
 (0.001, 1e-05, 0.3),
 (0.001, 1e-05, 0.4),
 (0.001, 1e-05, 0.5)]

In [ ]:
drop_out_rates = [0.1]#, 0.2, 0.3,0.4, 0.5
learning_rate_transfer_learnings = [1e-3, 1e-4]
learning_rate_fine_tunings = [1e-5, 1e-6]
albert_large = TFAlbertModel.from_pretrained('albert-large-v2')
for learning_rate_transfer_learning in learning_rate_transfer_learnings:
  for learning_rate_fine_tuning in learning_rate_fine_tunings:
    for drop_out_rate in drop_out_rates:
      if (learning_rate_transfer_learning, learning_rate_fine_tuning, drop_out_rate) not in seen_parameters:
        print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
        , learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)

        seen_parameters.append((learning_rate_transfer_learning, learning_rate_fine_tuning, drop_out_rate))

        #step1
        input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
        mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

        embeddings = albert_base(input_ids, attention_mask= mask)[0]
        X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
        X = tf.keras.layers.BatchNormalization()(X)
        X = tf.keras.layers.Dense(128, activation = 'relu')(X)
        X = tf.keras.layers.Dropout(drop_out_rate)(X)
        X = tf.keras.layers.Dense(32, activation = 'relu')(X)
        y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

        model2 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
        model2.layers[2].trainable = False
        #model2.summary()

        #step2
        optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
        loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
        metrics = []
        metrics.append(
            tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
        model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)
        #model2.summary() #Check trainable params increased.

        #step3: transfer learning
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
        history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

        #step4: predict
        balanced_acc=get_balanced_accuracy(model2)
        balanced_accuracies_transfer_learning.append( balanced_acc )

        #step5: fine tune
        model2.layers[2].trainable = True

        # It's important to recompile your model after you make any changes
        optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
        loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
        metrics = []
        metrics.append(
            tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
        model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)

        history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
        balanced_acc=get_balanced_accuracy(model2)
        balanced_accuracies_fine_tuning.append( balanced_acc )

        del(model2)

In [53]:
drop_out_rates = [0.2]#, 0.2, 0.3,0.4, 0.5
learning_rate_transfer_learnings = [1e-3, 1e-4]
learning_rate_fine_tunings = [1e-5, 1e-6]
albert_large = TFAlbertModel.from_pretrained('albert-large-v2')
for learning_rate_transfer_learning in learning_rate_transfer_learnings:
  for learning_rate_fine_tuning in learning_rate_fine_tunings:
    for drop_out_rate in drop_out_rates:
      if (learning_rate_transfer_learning, learning_rate_fine_tuning, drop_out_rate) not in seen_parameters:
        print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
        , learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)

        seen_parameters.append((learning_rate_transfer_learning, learning_rate_fine_tuning, drop_out_rate))

        #step1
        input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
        mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

        embeddings = albert_base(input_ids, attention_mask= mask)[0]
        X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
        X = tf.keras.layers.BatchNormalization()(X)
        X = tf.keras.layers.Dense(128, activation = 'relu')(X)
        X = tf.keras.layers.Dropout(drop_out_rate)(X)
        X = tf.keras.layers.Dense(32, activation = 'relu')(X)
        y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

        model2 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
        model2.layers[2].trainable = False
        #model2.summary()

        #step2
        optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
        loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
        metrics = []
        metrics.append(
            tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
        model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)
        #model2.summary() #Check trainable params increased.

        #step3: transfer learning
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
        history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

        #step4: predict
        balanced_acc=get_balanced_accuracy(model2)
        balanced_accuracies_transfer_learning.append( balanced_acc )

        #step5: fine tune
        model2.layers[2].trainable = True

        # It's important to recompile your model after you make any changes
        optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
        loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
        metrics = []
        metrics.append(
            tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
        model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)

        history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
        balanced_acc=get_balanced_accuracy(model2)
        balanced_accuracies_fine_tuning.append( balanced_acc )

        del(model2)

Some layers from the model checkpoint at albert-large-v2 were not used when initializing TFAlbertModel: ['predictions']
- This IS expected if you are initializing TFAlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFAlbertModel were initialized from the model checkpoint at albert-large-v2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertModel for predictions without further training.


learning_rate_transfer_learning:  0.001 learning_rate_fine_tuning:  1e-06 drop_out_rate: 0.2
Epoch 1/6
609/609 [==============================] - 246s 366ms/step - loss: 0.9767 - categorical_accuracy: 0.5497 - val_loss: 3.1171 - val_categorical_accuracy: 0.2730
Epoch 2/6
609/609 [==============================] - 218s 358ms/step - loss: 0.8618 - categorical_accuracy: 0.5754 - val_loss: 2.8146 - val_categorical_accuracy: 0.2941
Epoch 3/6
609/609 [==============================] - 218s 358ms/step - loss: 0.8139 - categorical_accuracy: 0.5956 - val_loss: 2.7685 - val_categorical_accuracy: 0.3079
Epoch 4/6
609/609 [==============================] - 218s 358ms/step - loss: 0.8014 - categorical_accuracy: 0.5962 - val_loss: 2.9234 - val_categorical_accuracy: 0.2950
Epoch 5/6
609/609 [==============================] - 218s 358ms/step - loss: 0.7832 - categorical_accuracy: 0.5935 - val_loss: 2.9021 - val_categorical_accuracy: 0.3033
Epoch 6/6
609/609 [==============================] - 218s 358m

609/609 [==============================] - 570s 890ms/step - loss: 0.7486 - categorical_accuracy: 0.6236 - val_loss: 3.1019 - val_categorical_accuracy: 0.2996
Epoch 2/6
609/609 [==============================] - 536s 881ms/step - loss: 0.6095 - categorical_accuracy: 0.6602 - val_loss: 3.2936 - val_categorical_accuracy: 0.2932
Epoch 3/6
609/609 [==============================] - 536s 881ms/step - loss: 0.5196 - categorical_accuracy: 0.6862 - val_loss: 3.3856 - val_categorical_accuracy: 0.3033
Epoch 4/6
609/609 [==============================] - 536s 881ms/step - loss: 0.4613 - categorical_accuracy: 0.7094 - val_loss: 3.5947 - val_categorical_accuracy: 0.2996
Epoch 5/6
609/609 [==============================] - 536s 881ms/step - loss: 0.4068 - categorical_accuracy: 0.7328 - val_loss: 3.7525 - val_categorical_accuracy: 0.3051
Epoch 6/6
609/609 [==============================] - 536s 881ms/step - loss: 0.3547 - categorical_accuracy: 0.7519 - val_loss: 3.9182 - val_categorical_accuracy: 0.3

609/609 [==============================] - 568s 889ms/step - loss: 0.6903 - categorical_accuracy: 0.6650 - val_loss: 2.6388 - val_categorical_accuracy: 0.2454
Epoch 2/6
609/609 [==============================] - 536s 880ms/step - loss: 0.8529 - categorical_accuracy: 0.6024 - val_loss: 2.5072 - val_categorical_accuracy: 0.3235
Epoch 3/6
609/609 [==============================] - 536s 881ms/step - loss: 0.6334 - categorical_accuracy: 0.6818 - val_loss: 2.9687 - val_categorical_accuracy: 0.2675
Epoch 4/6
609/609 [==============================] - 536s 880ms/step - loss: 0.6259 - categorical_accuracy: 0.6781 - val_loss: 2.8853 - val_categorical_accuracy: 0.2886
Epoch 5/6
609/609 [==============================] - 536s 881ms/step - loss: 0.6236 - categorical_accuracy: 0.6877 - val_loss: 2.8257 - val_categorical_accuracy: 0.1287


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.08      0.27      0.13       190
         1.0       0.18      0.13      0.15       173
         2.0       0.00      0.00      0.00       135
         3.0       0.17      0.45      0.25       216
         4.0       0.00      0.00      0.00      1036
         5.0       0.43      0.12      0.19        76
         6.0       0.94      0.52      0.67       195
         7.0       0.00      0.00      0.00       290
         8.0       0.05      0.46      0.09       139
         9.0       0.00      0.00      0.00       258

    accuracy                           0.13      2708
   macro avg       0.18      0.20      0.15      2708
weighted avg       0.11      0.13      0.10      2708

0.1951356969544379
learning_rate_transfer_learning:  0.0001 learning_rate_fine_tuning:  1e-06 drop_out_rate: 0.2
Epoch 1/6
609/609 [==============================] - 245s 367ms/step - loss: 1.2159 - categorical_accuracy: 0.4541 - val_loss: 2

609/609 [==============================] - 569s 893ms/step - loss: 0.7703 - categorical_accuracy: 0.5846 - val_loss: 2.5038 - val_categorical_accuracy: 0.2767
Epoch 2/6
609/609 [==============================] - 536s 881ms/step - loss: 0.5538 - categorical_accuracy: 0.6744 - val_loss: 2.7458 - val_categorical_accuracy: 0.2858
Epoch 3/6
609/609 [==============================] - 537s 881ms/step - loss: 0.4523 - categorical_accuracy: 0.7268 - val_loss: 2.7418 - val_categorical_accuracy: 0.2987
Epoch 4/6
609/609 [==============================] - 537s 881ms/step - loss: 0.3836 - categorical_accuracy: 0.7657 - val_loss: 2.9142 - val_categorical_accuracy: 0.3015
Epoch 5/6
609/609 [==============================] - 537s 881ms/step - loss: 0.3323 - categorical_accuracy: 0.7907 - val_loss: 2.9112 - val_categorical_accuracy: 0.3153
Epoch 6/6
609/609 [==============================] - 536s 881ms/step - loss: 0.2907 - categorical_accuracy: 0.8165 - val_loss: 3.0451 - val_categorical_accuracy: 0.3

In [54]:
balanced_accuracies_transfer_learning[5:]

[0.24114452614265702, 0.2493581581035141, 0.24538399803019706]

In [55]:
balanced_accuracies_fine_tuning[5:]

[0.2591786542521592, 0.1951356969544379, 0.24465893332573913]

In [ ]:
drop_out_rates = [0.3]#, 0.2, 0.3,0.4, 0.5
learning_rate_transfer_learnings = [1e-3, 1e-4]
learning_rate_fine_tunings = [1e-5, 1e-6]
albert_large = TFAlbertModel.from_pretrained('albert-large-v2')
for learning_rate_transfer_learning in learning_rate_transfer_learnings:
  for learning_rate_fine_tuning in learning_rate_fine_tunings:
    for drop_out_rate in drop_out_rates:
      if (learning_rate_transfer_learning, learning_rate_fine_tuning, drop_out_rate) not in seen_parameters:
        print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
        , learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)

        seen_parameters.append((learning_rate_transfer_learning, learning_rate_fine_tuning, drop_out_rate))

        #step1
        input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
        mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

        embeddings = albert_base(input_ids, attention_mask= mask)[0]
        X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
        X = tf.keras.layers.BatchNormalization()(X)
        X = tf.keras.layers.Dense(128, activation = 'relu')(X)
        X = tf.keras.layers.Dropout(drop_out_rate)(X)
        X = tf.keras.layers.Dense(32, activation = 'relu')(X)
        y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

        model2 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
        model2.layers[2].trainable = False
        #model2.summary()

        #step2
        optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
        loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
        metrics = []
        metrics.append(
            tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
        model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)
        #model2.summary() #Check trainable params increased.

        #step3: transfer learning
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
        history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

        #step4: predict
        balanced_acc=get_balanced_accuracy(model2)
        balanced_accuracies_transfer_learning.append( balanced_acc )

        #step5: fine tune
        model2.layers[2].trainable = True

        # It's important to recompile your model after you make any changes
        optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
        loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
        metrics = []
        metrics.append(
            tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
        model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)

        history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
        balanced_acc=get_balanced_accuracy(model2)
        balanced_accuracies_fine_tuning.append( balanced_acc )

        del(model2)

Some layers from the model checkpoint at albert-large-v2 were not used when initializing TFAlbertModel: ['predictions']
- This IS expected if you are initializing TFAlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFAlbertModel were initialized from the model checkpoint at albert-large-v2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertModel for predictions without further training.


learning_rate_transfer_learning:  0.001 learning_rate_fine_tuning:  1e-06 drop_out_rate: 0.3
Epoch 1/6
609/609 [==============================] - 245s 366ms/step - loss: 0.3530 - categorical_accuracy: 0.8181 - val_loss: 4.6522 - val_categorical_accuracy: 0.3079
Epoch 2/6
609/609 [==============================] - 218s 358ms/step - loss: 0.2819 - categorical_accuracy: 0.8460 - val_loss: 4.7302 - val_categorical_accuracy: 0.3235
Epoch 3/6
609/609 [==============================] - 218s 358ms/step - loss: 0.2534 - categorical_accuracy: 0.8581 - val_loss: 5.0115 - val_categorical_accuracy: 0.3502
Epoch 4/6
609/609 [==============================] - 218s 358ms/step - loss: 0.2191 - categorical_accuracy: 0.8698 - val_loss: 4.8203 - val_categorical_accuracy: 0.3373
Epoch 5/6
609/609 [==============================] - 218s 358ms/step - loss: 0.2138 - categorical_accuracy: 0.8627 - val_loss: 4.8558 - val_categorical_accuracy: 0.3125
Epoch 6/6
609/609 [==============================] - 218s 358m

609/609 [==============================] - 571s 892ms/step - loss: 0.2203 - categorical_accuracy: 0.8519 - val_loss: 5.0364 - val_categorical_accuracy: 0.3125
Epoch 2/6
609/609 [==============================] - 537s 882ms/step - loss: 0.1989 - categorical_accuracy: 0.8660 - val_loss: 5.1490 - val_categorical_accuracy: 0.3254
Epoch 3/6
609/609 [==============================] - 537s 882ms/step - loss: 0.1790 - categorical_accuracy: 0.8784 - val_loss: 5.3023 - val_categorical_accuracy: 0.3364
Epoch 4/6
609/609 [==============================] - 537s 882ms/step - loss: 0.1582 - categorical_accuracy: 0.8874 - val_loss: 5.3419 - val_categorical_accuracy: 0.3272
Epoch 5/6
609/609 [==============================] - 537s 882ms/step - loss: 0.1584 - categorical_accuracy: 0.8960 - val_loss: 5.2012 - val_categorical_accuracy: 0.3566
Epoch 6/6
609/609 [==============================] - 537s 882ms/step - loss: 0.1483 - categorical_accuracy: 0.9050 - val_loss: 4.9733 - val_categorical_accuracy: 0.3

609/609 [==============================] - 572s 892ms/step - loss: 0.4364 - categorical_accuracy: 0.7943 - val_loss: 3.5063 - val_categorical_accuracy: 0.3244
Epoch 2/6
609/609 [==============================] - 537s 882ms/step - loss: 2.0401 - categorical_accuracy: 0.2666 - val_loss: 2.3408 - val_categorical_accuracy: 0.0947
Epoch 3/6
609/609 [==============================] - 537s 882ms/step - loss: 1.9899 - categorical_accuracy: 0.2219 - val_loss: 2.0768 - val_categorical_accuracy: 0.1985
Epoch 4/6
609/609 [==============================] - 537s 882ms/step - loss: 1.7435 - categorical_accuracy: 0.2836 - val_loss: 2.0746 - val_categorical_accuracy: 0.2206


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       190
         1.0       0.15      0.28      0.20       173
         2.0       0.05      0.01      0.02       135
         3.0       0.11      0.23      0.15       216
         4.0       0.39      0.13      0.20      1036
         5.0       0.41      0.12      0.18        76
         6.0       0.76      0.76      0.76       195
         7.0       0.00      0.00      0.00       290
         8.0       0.10      0.12      0.11       139
         9.0       0.14      0.62      0.23       258

    accuracy                           0.21      2708
   macro avg       0.21      0.23      0.19      2708
weighted avg       0.26      0.21      0.19      2708

0.22925599574452962
learning_rate_transfer_learning:  0.0001 learning_rate_fine_tuning:  1e-06 drop_out_rate: 0.3
Epoch 1/6
609/609 [==============================] - 246s 368ms/step - loss: 1.5823 - categorical_accuracy: 0.2958 - val_loss: 

609/609 [==============================] - 571s 891ms/step - loss: 1.3501 - categorical_accuracy: 0.3499 - val_loss: 2.3684 - val_categorical_accuracy: 0.1967
Epoch 2/6


#### Resume from the last checkpoint

In [45]:
seen_parameters.append((1e-3, 1e-6, 0.3))
seen_parameters.append((1e-4, 1e-5, 0.3))
seen_parameters

[(0.001, 1e-05, 0.1),
 (0.001, 1e-05, 0.2),
 (0.001, 1e-05, 0.3),
 (0.001, 1e-05, 0.4),
 (0.001, 1e-05, 0.5),
 (0.001, 1e-06, 0.3),
 (0.0001, 1e-05, 0.3)]

In [52]:
seen_parameters.remove((0.0001, 1e-06, 0.3))
seen_parameters

[(0.001, 1e-05, 0.1),
 (0.001, 1e-05, 0.2),
 (0.001, 1e-05, 0.3),
 (0.001, 1e-05, 0.4),
 (0.001, 1e-05, 0.5),
 (0.001, 1e-06, 0.3),
 (0.0001, 1e-05, 0.3)]

In [47]:
McNemar = {}
balanced_accuracies_transfer_learning = []
balanced_accuracies_fine_tuning = []

In [53]:
drop_out_rates = [0.3]#, 0.2, 0.3,0.4, 0.5
learning_rate_transfer_learnings = [1e-3, 1e-4]
learning_rate_fine_tunings = [1e-5, 1e-6]
albert_large = TFAlbertModel.from_pretrained('albert-large-v2')
for learning_rate_transfer_learning in learning_rate_transfer_learnings:
  for learning_rate_fine_tuning in learning_rate_fine_tunings:
    for drop_out_rate in drop_out_rates:
      if (learning_rate_transfer_learning, learning_rate_fine_tuning, drop_out_rate) not in seen_parameters:
        print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
        , learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)

        seen_parameters.append((learning_rate_transfer_learning, learning_rate_fine_tuning, drop_out_rate))

        #step1
        input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
        mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

        embeddings = albert_large(input_ids, attention_mask= mask)[0]
        X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
        X = tf.keras.layers.BatchNormalization()(X)
        X = tf.keras.layers.Dense(128, activation = 'relu')(X)
        X = tf.keras.layers.Dropout(drop_out_rate)(X)
        X = tf.keras.layers.Dense(32, activation = 'relu')(X)
        y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

        model2 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
        model2.layers[2].trainable = False
        #model2.summary()

        #step2
        optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
        loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
        metrics = []
        metrics.append(
            tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
        model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)
        #model2.summary() #Check trainable params increased.

        #step3: transfer learning
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
        history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

        #step4: predict
        balanced_acc, McNemar=get_balanced_accuracy(model2, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
        balanced_accuracies_transfer_learning.append( balanced_acc )

        #step5: fine tune
        model2.layers[2].trainable = True

        # It's important to recompile your model after you make any changes
        optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
        loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
        metrics = []
        metrics.append(
            tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
        model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)

        history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
        balanced_acc, McNemar=get_balanced_accuracy(model2, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
        balanced_accuracies_fine_tuning.append( balanced_acc )

        del(model2)

Some layers from the model checkpoint at albert-large-v2 were not used when initializing TFAlbertModel: ['predictions']
- This IS expected if you are initializing TFAlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFAlbertModel were initialized from the model checkpoint at albert-large-v2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertModel for predictions without further training.


learning_rate_transfer_learning:  0.0001 learning_rate_fine_tuning:  1e-06 drop_out_rate: 0.3
Epoch 1/6
609/609 [==============================] - 418s 648ms/step - loss: 2.3861 - categorical_accuracy: 0.1485 - val_loss: 2.1801 - val_categorical_accuracy: 0.1912
Epoch 2/6
609/609 [==============================] - 390s 640ms/step - loss: 2.1133 - categorical_accuracy: 0.2192 - val_loss: 2.0884 - val_categorical_accuracy: 0.2270
Epoch 3/6
609/609 [==============================] - 389s 639ms/step - loss: 1.9941 - categorical_accuracy: 0.2608 - val_loss: 2.0360 - val_categorical_accuracy: 0.2629
Epoch 4/6
609/609 [==============================] - 389s 640ms/step - loss: 1.9020 - categorical_accuracy: 0.2973 - val_loss: 1.9806 - val_categorical_accuracy: 0.2785
Epoch 5/6
609/609 [==============================] - 389s 639ms/step - loss: 1.8229 - categorical_accuracy: 0.3197 - val_loss: 1.9525 - val_categorical_accuracy: 0.3024
Epoch 6/6
609/609 [==============================] - 389s 639

609/609 [==============================] - 986s 2s/step - loss: 1.7809 - categorical_accuracy: 0.3281 - val_loss: 1.9469 - val_categorical_accuracy: 0.2978
Epoch 2/6
609/609 [==============================] - 956s 2s/step - loss: 1.7997 - categorical_accuracy: 0.3209 - val_loss: 1.9459 - val_categorical_accuracy: 0.2886
Epoch 3/6
609/609 [==============================] - 956s 2s/step - loss: 1.8179 - categorical_accuracy: 0.3180 - val_loss: 1.9257 - val_categorical_accuracy: 0.2969
Epoch 4/6
609/609 [==============================] - 956s 2s/step - loss: 1.7903 - categorical_accuracy: 0.3312 - val_loss: 1.9016 - val_categorical_accuracy: 0.3006
Epoch 5/6
609/609 [==============================] - 956s 2s/step - loss: 1.7939 - categorical_accuracy: 0.3264 - val_loss: 1.9550 - val_categorical_accuracy: 0.2932
Epoch 6/6
609/609 [==============================] - 956s 2s/step - loss: 1.7994 - categorical_accuracy: 0.3276 - val_loss: 1.9756 - val_categorical_accuracy: 0.2757
              

In [55]:
balanced_accuracies_fine_tuning

[0.2871389490893207]

In [58]:
drop_out_rates = [0.4]#, 0.2, 0.3,0.4, 0.5
learning_rate_transfer_learnings = [1e-3, 1e-4]
learning_rate_fine_tunings = [1e-5, 1e-6]
albert_large = TFAlbertModel.from_pretrained('albert-large-v2')
for learning_rate_transfer_learning in learning_rate_transfer_learnings:
  for learning_rate_fine_tuning in learning_rate_fine_tunings:
    for drop_out_rate in drop_out_rates:
      if (learning_rate_transfer_learning, learning_rate_fine_tuning, drop_out_rate) not in seen_parameters:
        print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
        , learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)

        seen_parameters.append((learning_rate_transfer_learning, learning_rate_fine_tuning, drop_out_rate))

        #step1
        input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
        mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

        embeddings = albert_large(input_ids, attention_mask= mask)[0]
        X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
        X = tf.keras.layers.BatchNormalization()(X)
        X = tf.keras.layers.Dense(128, activation = 'relu')(X)
        X = tf.keras.layers.Dropout(drop_out_rate)(X)
        X = tf.keras.layers.Dense(32, activation = 'relu')(X)
        y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

        model2 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
        model2.layers[2].trainable = False
        #model2.summary()

        #step2
        optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
        loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
        metrics = []
        metrics.append(
            tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
        model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)
        #model2.summary() #Check trainable params increased.

        #step3: transfer learning
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
        history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

        #step4: predict
        balanced_acc, McNemar=get_balanced_accuracy(model2, McNemar, False, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
        balanced_accuracies_transfer_learning.append( balanced_acc )

        #step5: fine tune
        model2.layers[2].trainable = True

        # It's important to recompile your model after you make any changes
        optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
        loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
        metrics = []
        metrics.append(
            tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
        model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)

        history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
        balanced_acc, McNemar=get_balanced_accuracy(model2, McNemar, True, drop_out_rate,learning_rate_transfer_learning, learning_rate_fine_tuning )
        balanced_accuracies_fine_tuning.append( balanced_acc )

        del(model2)

Some layers from the model checkpoint at albert-large-v2 were not used when initializing TFAlbertModel: ['predictions']
- This IS expected if you are initializing TFAlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFAlbertModel were initialized from the model checkpoint at albert-large-v2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertModel for predictions without further training.


learning_rate_transfer_learning:  0.0001 learning_rate_fine_tuning:  1e-06 drop_out_rate: 0.4
Epoch 1/6
609/609 [==============================] - 418s 648ms/step - loss: 2.5535 - categorical_accuracy: 0.1344 - val_loss: 2.2163 - val_categorical_accuracy: 0.1912
Epoch 2/6
609/609 [==============================] - 389s 639ms/step - loss: 2.2077 - categorical_accuracy: 0.1970 - val_loss: 2.1123 - val_categorical_accuracy: 0.2022
Epoch 3/6
609/609 [==============================] - 390s 640ms/step - loss: 2.0754 - categorical_accuracy: 0.2191 - val_loss: 2.0452 - val_categorical_accuracy: 0.2197
Epoch 4/6
609/609 [==============================] - 390s 640ms/step - loss: 1.9700 - categorical_accuracy: 0.2524 - val_loss: 2.0080 - val_categorical_accuracy: 0.2215
Epoch 5/6
609/609 [==============================] - 389s 639ms/step - loss: 1.9136 - categorical_accuracy: 0.2634 - val_loss: 1.9795 - val_categorical_accuracy: 0.2307
Epoch 6/6
609/609 [==============================] - 389s 639

609/609 [==============================] - 988s 2s/step - loss: 1.8312 - categorical_accuracy: 0.2922 - val_loss: 1.9666 - val_categorical_accuracy: 0.2371
Epoch 2/6
609/609 [==============================] - 956s 2s/step - loss: 1.8744 - categorical_accuracy: 0.2819 - val_loss: 1.9689 - val_categorical_accuracy: 0.2426
Epoch 3/6
609/609 [==============================] - 957s 2s/step - loss: 1.8781 - categorical_accuracy: 0.2731 - val_loss: 1.9869 - val_categorical_accuracy: 0.2491
Epoch 4/6
609/609 [==============================] - 956s 2s/step - loss: 1.8775 - categorical_accuracy: 0.2720 - val_loss: 1.9427 - val_categorical_accuracy: 0.2702
Epoch 5/6
609/609 [==============================] - 956s 2s/step - loss: 1.8841 - categorical_accuracy: 0.2818 - val_loss: 1.9828 - val_categorical_accuracy: 0.2399
Epoch 6/6
609/609 [==============================] - 955s 2s/step - loss: 1.8930 - categorical_accuracy: 0.2762 - val_loss: 1.9521 - val_categorical_accuracy: 0.2693
              

In [ ]:
drop_out_rates = [0.5]#, 0.2, 0.3,0.4, 0.5
learning_rate_transfer_learnings = [1e-3, 1e-4]
learning_rate_fine_tunings = [1e-5, 1e-6]
albert_large = TFAlbertModel.from_pretrained('albert-large-v2')
for learning_rate_transfer_learning in learning_rate_transfer_learnings:
  for learning_rate_fine_tuning in learning_rate_fine_tunings:
    for drop_out_rate in drop_out_rates:
      if (learning_rate_transfer_learning, learning_rate_fine_tuning, drop_out_rate) not in seen_parameters:
        print("learning_rate_transfer_learning: ",learning_rate_transfer_learning, "learning_rate_fine_tuning: "
        , learning_rate_fine_tuning, "drop_out_rate:", drop_out_rate)

        seen_parameters.append((learning_rate_transfer_learning, learning_rate_fine_tuning, drop_out_rate))

        #step1
        input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
        mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

        embeddings = albert_base(input_ids, attention_mask= mask)[0]
        X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
        X = tf.keras.layers.BatchNormalization()(X)
        X = tf.keras.layers.Dense(128, activation = 'relu')(X)
        X = tf.keras.layers.Dropout(drop_out_rate)(X)
        X = tf.keras.layers.Dense(32, activation = 'relu')(X)
        y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

        model2 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
        model2.layers[2].trainable = False
        #model2.summary()

        #step2
        optimizer = tf.keras.optimizers.Adam(learning_rate_transfer_learning)
        loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
        metrics = []
        metrics.append(
            tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
        model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)
        #model2.summary() #Check trainable params increased.

        #step3: transfer learning
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
        history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

        #step4: predict
        balanced_acc=get_balanced_accuracy(model2)
        balanced_accuracies_transfer_learning.append( balanced_acc )

        #step5: fine tune
        model2.layers[2].trainable = True

        # It's important to recompile your model after you make any changes
        optimizer = tf.keras.optimizers.Adam(learning_rate_fine_tuning)
        loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
        metrics = []
        metrics.append(
            tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
        model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)

        history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
        balanced_acc=get_balanced_accuracy(model2)
        balanced_accuracies_fine_tuning.append( balanced_acc )

        del(model2)

From this result, ALBERT performs best at the dropout rate of ____.

### KFolds

In [44]:
def get_Xid_Xmask(X_origin):
  Xids_train = np.zeros((X_origin.shape[0], SEQ_LEN))
  Xmask_train = np.zeros((X_origin.shape[0], SEQ_LEN))

  for i, lyric in enumerate(X_origin):
    tokens = tokenizer.encode_plus(lyric, max_length=SEQ_LEN, truncation =True, padding="max_length", add_special_tokens = True, return_token_type_ids= False, return_attention_mask = True, return_tensors= 'tf')
    Xids_train[i,:], Xmask_train[i,:] = tokens['input_ids'], tokens['attention_mask']
  return Xids_train, Xmask_train

In [ ]:
BEST_DROPOUT_RATE = 

In [ ]:
balanced_accuracies = []
for i in range(1,5):
  #step1: train, val, test preparation.
  split0=splits[i]
  split0['X_train'] = prepare_lyrics(split0['X_train'] )
  split0['X_test'] = prepare_lyrics(split0['X_test'] )
  #tokenizer = AlbertTokenizer.from_pretrained('albert-large-v2')
  Xids_train, Xmask_train = get_Xid_Xmask(split0['X_train'])
  Xids_test, Xmask_test = get_Xid_Xmask(split0['X_test'])

  labels_train = np.zeros((split0['y_train'].shape[0], 10))
  labels_train[ np.arange(split0['y_train'].shape[0]), split0['y_train'].values] =1
  labels_test = np.zeros((split0['y_test'].shape[0], 10))
  labels_test[ np.arange(split0['y_test'].shape[0]), split0['y_test'].values] =1

  dataset_train = tf.data.Dataset.from_tensor_slices((Xids_train, Xmask_train, labels_train))
  dataset_test = tf.data.Dataset.from_tensor_slices((Xids_test, Xmask_test, labels_test))
  dataset_train = dataset_train.map(map_func)
  dataset_test = dataset_test.map(map_func)

  dataset_train = dataset_train.shuffle(42).batch(16)
  train = dataset_train.take(round(DS_LEN*SPLIT))
  val = dataset_train.skip(round(DS_LEN*SPLIT))
  test = dataset_test.batch(16)

  #step2: build a ALBERT model
  input_ids = tf.keras.layers.Input(shape=(SEQ_LEN2,), name= 'input_ids', dtype='int32')
  mask = tf.keras.layers.Input(shape=(SEQ_LEN2,), name='attention_mask')

  embeddings = albert_base(input_ids, attention_mask= mask)[0]
  X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
  X = tf.keras.layers.BatchNormalization()(X)
  X = tf.keras.layers.Dense(128, activation = 'relu')(X)
  X = tf.keras.layers.Dropout(BEST_DROPOUT_RATE)(X)
  X = tf.keras.layers.Dense(32, activation = 'relu')(X)
  y= tf.keras.layers.Dense(10, activation = 'softmax' , name= 'outputs')(X)

  model2 = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
  model2.layers[2].trainable = False

  #step2
  optimizer = tf.keras.optimizers.Adam(1e-3)
  loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
  metrics = []
  metrics.append(
      tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
  model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)
  #model2.summary() #Check trainable params increased.

  #step3: transfer learning
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)
  history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])

  #step4: predict
  y_pred_transfer_learning=get_balanced_accuracy(model2)

  #step5: fine tune
  model2.layers[2].trainable = True

  # It's important to recompile your model after you make any changes
  optimizer = tf.keras.optimizers.Adam(1e-5)
  loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
  metrics = []
  metrics.append(
      tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
  model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)

  history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])
  y_pred_transfer_learning=get_balanced_accuracy(model2)

  del(model2)



### Unnecessary codes

In [ ]:
print(len(train), len(val), len(test))

609 68 170


First, transfer learning

In [ ]:
optimizer = tf.keras.optimizers.Adam(1e-3)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)
model2.summary() #Check trainable params increased.

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_albert_model (TFAlbertModel  TFBaseModelOutputWi  11683584   ['input_ids[0][0]',              
 )                              thPooling(last_hidd               'attention_mask[0][0]']         
                                en_state=(None, 256                                               
                                , 768),                                                       

In [ ]:
VAL_SIZE=1088

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3)

#metrics = Metrics()
history = model2.fit(train, validation_data=val, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])#metrics
#metrics.get_data()

Epoch 1/6
609/609 [==============================] - 90s 128ms/step - loss: 2.1484 - categorical_accuracy: 0.2435 - val_loss: 1.9851 - val_categorical_accuracy: 0.2776
Epoch 2/6
609/609 [==============================] - 75s 124ms/step - loss: 1.8344 - categorical_accuracy: 0.3179 - val_loss: 1.9732 - val_categorical_accuracy: 0.2794
Epoch 3/6
609/609 [==============================] - 76s 124ms/step - loss: 1.6454 - categorical_accuracy: 0.3703 - val_loss: 1.9635 - val_categorical_accuracy: 0.2978
Epoch 4/6
609/609 [==============================] - 75s 124ms/step - loss: 1.4627 - categorical_accuracy: 0.4187 - val_loss: 2.0251 - val_categorical_accuracy: 0.3024
Epoch 5/6
609/609 [==============================] - 75s 124ms/step - loss: 1.2999 - categorical_accuracy: 0.4579 - val_loss: 2.1355 - val_categorical_accuracy: 0.2868
Epoch 6/6
609/609 [==============================] - 75s 124ms/step - loss: 1.1250 - categorical_accuracy: 0.5019 - val_loss: 2.1858 - val_categorical_accuracy:

In [ ]:
TEST_SIZE = split0["y_test"].shape[0]
TEST_SIZE

2708

In [ ]:
y_pred_transfer_learning=get_balanced_accuracy(model2)

              precision    recall  f1-score   support

         0.0       0.16      0.24      0.19       190
         1.0       0.17      0.27      0.21       173
         2.0       0.12      0.10      0.11       135
         3.0       0.14      0.17      0.15       216
         4.0       0.66      0.31      0.42      1036
         5.0       0.13      0.17      0.15        76
         6.0       0.54      0.72      0.62       195
         7.0       0.14      0.07      0.09       290
         8.0       0.09      0.25      0.13       139
         9.0       0.19      0.28      0.23       258

    accuracy                           0.28      2708
   macro avg       0.23      0.26      0.23      2708
weighted avg       0.37      0.28      0.29      2708

0.25892839922093847


Fine tuning with smaller learning rate

In [ ]:
# Unfreeze the base model
#model.trainable = True
model2.layers[2].trainable = True

# It's important to recompile your model after you make any changes
optimizer = tf.keras.optimizers.Adam(1e-5)
loss = tf.keras.losses.CategoricalCrossentropy(name='categorical_crossentropy')#from_logits=False,label_smoothing=0.0,axis=-1,
metrics = []
metrics.append(
    tf.metrics.CategoricalAccuracy(name='categorical_accuracy', dtype=None))
model2.compile(optimizer=optimizer, loss= loss, metrics=metrics)
model2.summary() #Check trainable params increased.

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_roberta_model (TFRobertaMod  TFBaseModelOutputWi  124645632  ['input_ids[0][0]',              
 el)                            thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

In [ ]:
#metrics = Metrics()
history = model2.fit(train2, validation_data=val2, epochs=6, class_weight=my_weight2 ,callbacks=[ early_stopping])#metrics
#metrics.get_data()

Epoch 1/6


609/609 [==============================] - 203s 307ms/step - loss: 1.7488 - categorical_accuracy: 0.3503 - val_loss: 1.9209 - val_categorical_accuracy: 0.3309
Epoch 2/6
609/609 [==============================] - 185s 303ms/step - loss: 1.6219 - categorical_accuracy: 0.3758 - val_loss: 1.8664 - val_categorical_accuracy: 0.3373
Epoch 3/6
609/609 [==============================] - 185s 303ms/step - loss: 1.5014 - categorical_accuracy: 0.4157 - val_loss: 1.8434 - val_categorical_accuracy: 0.2960
Epoch 4/6
609/609 [==============================] - 185s 304ms/step - loss: 1.4113 - categorical_accuracy: 0.4380 - val_loss: 1.8316 - val_categorical_accuracy: 0.3373
Epoch 5/6
609/609 [==============================] - 185s 303ms/step - loss: 1.2980 - categorical_accuracy: 0.4698 - val_loss: 1.8991 - val_categorical_accuracy: 0.3419
Epoch 6/6
609/609 [==============================] - 185s 304ms/step - loss: 1.2123 - categorical_accuracy: 0.5004 - val_loss: 1.7828 - val_categorical_accuracy: 0.3

In [ ]:
get_balanced_accuracy(model2)

              precision    recall  f1-score   support

         0.0       0.26      0.23      0.24       190
         1.0       0.19      0.32      0.24       173
         2.0       0.09      0.25      0.13       135
         3.0       0.25      0.20      0.22       216
         4.0       0.74      0.41      0.53      1036
         5.0       0.23      0.41      0.30        76
         6.0       0.65      0.90      0.76       195
         7.0       0.14      0.01      0.01       290
         8.0       0.13      0.23      0.17       139
         9.0       0.27      0.47      0.34       258

    accuracy                           0.36      2708
   macro avg       0.30      0.34      0.29      2708
weighted avg       0.44      0.36      0.36      2708

0.34310539549600694
